In [37]:
import pandas as pd
import numpy as np
import altair as alt
import requests
import qgrid
import networkx as nx

qgrid.enable()

## Scrape Eurovision vote data

In [17]:
proper_names = ["Albania", "Armenia", "Australia", "Austria", 
                "Azerbaijan", "Belarus", "Belgium", "Bulgaria", 
                "Croatia", "Cyprus", "Czech Republic", "Denmark", 
                "Estonia", "F.Y.R. Macedonia", "Finland", "France", 
                "Georgia", "Germany", "Greece", "Hungary", 
                "Iceland", "Ireland", "Israel", "Italy", "Latvia", 
                "Lithuania", "Malta", "Moldova", "Montenegro", 
                "Norway", "Poland", "Portugal", "Romania", 
                "Russia", "San Marino", "Serbia", "Slovenia", 
                "Spain", "Sweden", "Switzerland", "The Netherlands", 
                "Ukraine", "United Kingdom"]

countries = ["albania", "armenia", "australia", "austria", 
             "azerbaijan", "belarus", "belgium", "bulgaria", 
             "croatia", "cyprus", "czech-republic", "denmark", 
             "estonia", "fyr-macedonia", "finland", "france", 
             "georgia", "germany", "greece", "hungary", 
             "iceland", "ireland", "israel", "italy", "latvia", 
             "lithuania", "malta", "moldova", "montenegro", 
             "norway", "poland", "portugal", "romania", 
             "russia", "san-marino", "serbia", "slovenia", 
             "spain", "sweden", "switzerland", "the-netherlands", 
             "ukraine", "united-kingdom"]

In [13]:
url = 'https://eurovision.tv/event/lisbon-2018/grand-final/voting-details/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

dfs = []

for c in countries:
    r = requests.get(url+c, headers=header)
    dfs.extend(pd.read_html(r.text, match='Participants'))

## Append to on DataFrame

In [66]:

def to_numeric(df):
    '''
    Turn timedelta columns into numeric dtype
    '''
    cols = ['A', 'B', 'C', 'D', 'E', 'Jury rank', 'Jury points',
           'Televoting rank', 'Televoting points']
    numeric = df[cols].apply(pd.to_numeric, errors='coerce')
    df = df.copy()
    df[cols] = numeric
    return df

df = (pd.concat(dfs, keys=proper_names)
             .pipe(to_numeric))

## Calculate "antisemitism"

In [119]:
qgrid.set_grid_option('maxVisibleRows', 100)

(df[['Participants', 'A', 'B', 'C', 'D', 'E']][df.Participants=='Israel']
 .assign(jury_avg_rank = lambda x: (x.A+x.B+x.C+x.D+x.E)/5)
 .assign(Antisemitism = lambda x: x.jury_avg_rank.rank(ascending=False))
 .reset_index()[['level_0', 'jury_avg_rank', 'Antisemitism']]
 .rename(columns={'level_0': 'Country', 'jury_avg_rank': 'Jury Average Ranking'}).set_index('Country')
)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 100, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5)

In [123]:
(df[['Participants', 'A', 'B', 'C', 'D', 'E']][df.Participants=='Israel']
 .assign(jury_avg_rank = lambda x: (x.A+x.B+x.C+x.D+x.E)/5)
 .assign(Antisemitism = lambda x: x.jury_avg_rank.rank(ascending=False, method='min'))
 .reset_index()[['level_0', 'jury_avg_rank', 'Antisemitism', 'A', 'B', 'C', 'D', 'E']]
 .rename(columns={'level_0': 'Country', 'jury_avg_rank': 'Jury Average Ranking'}).set_index('Country'))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 100, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5)

In [120]:
(df[['Participants', 'A', 'B', 'C', 'D', 'E']][df.Participants=='Israel']
 .assign(jury_avg_rank = lambda x: (x.A+x.B+x.C+x.D+x.E)/5)
 .assign(Antisemitism = lambda x: x.jury_avg_rank.rank(ascending=False))
 .reset_index()[['level_0', 'jury_avg_rank', 'Antisemitism']]
 .rename(columns={'level_0': 'Country', 'jury_avg_rank': 'Jury Average Ranking'}).set_index('Country')
).to_csv('eurovision/antisemite.csv')

In [101]:
df.to_csv('eurovision/all.csv')

## Create adjacency matrix for graph stuff

In [99]:
def get_adj(df, weight_col, reindex=True):
    adj = (df.reset_index()[['level_0', 'Participants', weight_col]]
              .rename(columns={'level_0': 'from', 'Participants': 'to', weight_col: 'weight'})
              .pivot(index='from', columns='to', values='weight')
              )
    if reindex:
        adj = adj.reindex(labels=proper_names, axis=1)
    return adj

In [91]:
jury_ranks = get_adj(df, 'Jury rank')
jury_points = get_adj(df, 'Jury points')
tele = get_adj(df, 'Televoting points')


In [92]:
G = nx.from_pandas_adjacency(tele.fillna(0))
nx.write_gexf(G, 'eurovision/euro18_tele_points.gexf')

In [96]:
clean_names = dict(zip(proper_names, [c.replace(' ', '_').replace('.', '') for c in proper_names]))
clean_names

{'Albania': 'Albania',
 'Armenia': 'Armenia',
 'Australia': 'Australia',
 'Austria': 'Austria',
 'Azerbaijan': 'Azerbaijan',
 'Belarus': 'Belarus',
 'Belgium': 'Belgium',
 'Bulgaria': 'Bulgaria',
 'Croatia': 'Croatia',
 'Cyprus': 'Cyprus',
 'Czech Republic': 'Czech_Republic',
 'Denmark': 'Denmark',
 'Estonia': 'Estonia',
 'F.Y.R. Macedonia': 'FYR_Macedonia',
 'Finland': 'Finland',
 'France': 'France',
 'Georgia': 'Georgia',
 'Germany': 'Germany',
 'Greece': 'Greece',
 'Hungary': 'Hungary',
 'Iceland': 'Iceland',
 'Ireland': 'Ireland',
 'Israel': 'Israel',
 'Italy': 'Italy',
 'Latvia': 'Latvia',
 'Lithuania': 'Lithuania',
 'Malta': 'Malta',
 'Moldova': 'Moldova',
 'Montenegro': 'Montenegro',
 'Norway': 'Norway',
 'Poland': 'Poland',
 'Portugal': 'Portugal',
 'Romania': 'Romania',
 'Russia': 'Russia',
 'San Marino': 'San_Marino',
 'Serbia': 'Serbia',
 'Slovenia': 'Slovenia',
 'Spain': 'Spain',
 'Sweden': 'Sweden',
 'Switzerland': 'Switzerland',
 'The Netherlands': 'The_Netherlands',
 'Uk

In [97]:
tele.fillna('-').rename(columns=clean_names, index=clean_names).to_csv('eurovision/tele_circ.csv', sep='\t')

In [98]:
jury_points.fillna('-').rename(columns=clean_names, index=clean_names).to_csv('eurovision/jury_circ.csv', sep='\t')

In [100]:
tele_circ = get_adj(df, 'Televoting points', reindex=False)
jury_circ = get_adj(df, 'Jury points', reindex=False)
tele_circ.fillna('-').rename(columns=clean_names, index=clean_names).to_csv('eurovision/tele_circ.csv', sep='\t')
jury_circ.fillna('-').rename(columns=clean_names, index=clean_names).to_csv('eurovision/jury_circ.csv', sep='\t')